In [74]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import math

In [75]:
df = pd.read_csv("./dataset/20170827_mawilab_flow_000.csv",sep=',')
df.head() 

,sIP,dIP,sPort,dPort,pro,packets,bytes,flags,sTime,durat,...,appli,Records,%Records,cumul_%,status,taxonomy,label,heuristic,distance,nbDetectors
0,70.37.209.160,203.205.84.19,0,0,1,1,32,NaN,2017/08/27T05:00:00.290,0.0,...,0,normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,118.150.224.119,163.158.254.254,80,14521,6,1,48,S A,2017/08/27T05:00:00.290,0.0,...,0,normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,203.104.241.234,149.20.207.139,22,51316,6,1,1440,A,2017/08/27T05:00:00.290,0.0,...,0,normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,164.122.96.252,203.205.73.175,56670,80,6,1,52,A,2017/08/27T05:00:00.290,0.0,...,0,anomaly,mptpHTTP,Multi Points,0.39596,8.0,anomalous,NaN,NaN,NaN
4,164.122.96.252,203.205.73.175,56670,80,6,1,52,A,2017/08/27T05:00:00.290,0.0,...,0,anomaly,mptpHTTP,Multi Points,0.39596,8.0,anomalous,NaN,NaN,NaN


In [76]:
df.loc[:, 'U'] = pd.Series(np.zeros(df.shape[0]),dtype='int8')
df.loc[:, 'N'] = pd.Series(np.zeros(df.shape[0]),dtype='int8')
new_flag_df = df['flags'].to_frame()
new_flag_df = new_flag_df['flags'].str.get_dummies(' ')
new_flag_df.head()

,A,C,E,EC,F,PA,R,S
0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0


In [77]:
"""
1. Pro
2. Packets
3. Bytes
4. Flags
5. Durat
6. Records
7. svc_port - combination of sPort and dPort

v = np.minimum(df['sPort'], df['dPort'])
df['svc_port'] = np.where(v <= 1024, v, df['dPort'])
"""
df['svc_port'] = df.apply(lambda x : min(x['sPort'],x['dPort']) if min(x['sPort'],x['dPort']) <= 1024 else x['dPort'], axis=1)
df = df.loc[:,('pro', 'packets', 'bytes', 'svc_port', 'durat','U','N', 'Records')]
df.fillna(0, inplace=True) 
new_df = df.drop(['Records'], axis=1)
mawii_labels = pd.get_dummies(df['Records'],prefix='',prefix_sep='')
mawii_features = pd.concat([new_df, new_flag_df], axis=1)
print(mawii_labels.head())
print(mawii_features.head())

   anomaly  normal
0        0       1
1        0       1
2        0       1
3        1       0
4        1       0
   pro  packets  bytes  svc_port  durat  U  N  A  C  E  EC  F  PA  R  S
0    1        1     32         0    0.0  0  0  0  0  0   0  0   0  0  0
1    6        1     48        80    0.0  0  0  1  0  0   0  0   0  0  1
2    6        1   1440        22    0.0  0  0  1  0  0   0  0   0  0  0
3    6        1     52        80    0.0  0  0  1  0  0   0  0   0  0  0
4    6        1     52        80    0.0  0  0  1  0  0   0  0   0  0  0


In [78]:
mawii_features = mawii_features.as_matrix()
mawii_labels = mawii_labels.as_matrix()
import pickle 
mawii_features.dump("mawi_features.pkl")
mawii_labels.dump("mawi_labels.pkl")

In [66]:
"""
# Converting to Tensor: 
mawii_features = tf.convert_to_tensor(mawii_features, dtype=)
mawii_labels = tf.convert_to_tensor(mawii_labels)
"""

'\n# Converting to Tensor: \nmawii_features = tf.convert_to_tensor(mawii_features, dtype=)\nmawii_labels = tf.convert_to_tensor(mawii_labels)\n'

In [73]:

in_width = 15 
mawii_features = tf.reshape(mawii_features, [1, in_width, 1])
mawii_features.shape



ValueError: Cannot reshape a tensor with 5754225 elements to shape [1,15,1] (15 elements) for 'Reshape_14' (op: 'Reshape') with input shapes: [1,1,383615,15], [3].

In [68]:
def conv_1d(x,W): 
    # Padding : surround the original image with 0's 
    # valid padding - no padding.size of the image shrinks  resultant image : (n - f + 1, n - f + 1)
    # same padding - resultant is same as that of original. resultant image : (n + 2p - f + 1, n +2p - f + 1)  p = (f - 1)/2
    # Strides : ((n + 2p - f)/s + 1, (n +2p - f)/s + 1)
    return tf.nn.conv1d(input = x, filter = W, strides =1, padding="SAME")

def avg_pool_1x1(x):
    # MAX/AVG Pooling : ((n + 2p - f)/s + 1, (n +2p - f)/s + 1)
    return tf.nn.avg_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

In [69]:
# cnn-1,2,3 model 

In [70]:
def conv1_network(x_image, reuse=False): 
    with tf.variable_scope('ConvNet_1') as scope: 
        if reuse:
             tf.get_variable_scope().reuse_variables()
        # First Conv and Pooling layers
        W_conv1 = tf.get_variable('c_wconv1',[-1,,1],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv1 = tf.get_variable('c_bconv1',[8],initializer=tf.constant_initializer(0))
        h_conv1 = tf.nn.relu(conv_1d(x_image, W_conv1) + b_conv1)
        h_pool1 = avg_pool_2x2(h_conv1)
        print(h_pool1.shape)
        
        # Fully Connected layer1 
        # First fully connected layers
        W_fc1 = tf.get_variable('d_wfc1', [1534464,16],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc1 = tf.get_variable('d_bfc1', [16], initializer=tf.constant_initializer(0))
        h_pool_flat = tf.reshape(h_pool1, [-1,1534464])
        print(tf.shape(h_pool_flat))
        h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat,W_fc1)+b_fc1)
        
        #Seconf fully connected layers
        W_fc2 = tf.get_variable('d_wfc2', [16,2],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc2 = tf.get_variable('d_bfc2', [2], initializer=tf.constant_initializer(0))

        y_conv = tf.nn.relu(tf.matmul(h_fc1,W_fc2)+b_fc2)
        print(y_conv.shape)
    
    return tf.reshape(y_conv, [mawii_features.shape[2],2])

In [71]:
def batches(batch_size, features, labels): 
    assert len(features) == len(labels)
    sample_size = len(features)
    output_batches = [] 
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        output_batches.append(batch)
    return output_batches
def print_epochs_cost_accuracy(epoch, sess, batch_features, batch_labels): 
    costVal = sess.run(cost, feed_dict = {features: batch_features, labels:batch_labels})
    AccuracyVal = sess.run(accuracy,feed_dict = {features: valid_features, labels : valid_labels})
    print("epoch {}, cost {}, Accuracy {} ".format(epoch, costVal, AccuracyVal))

In [72]:
-

(1, 1, ?, 8)
Tensor("ConvNet_1_5/Shape:0", shape=(2,), dtype=int32)
(?, 2)


InvalidArgumentError: Input to reshape is a tensor with 2 values, but the requested shape has 767230
	 [[Node: Reshape_10 = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](ConvNet_1_5/Relu_2, Reshape_10/shape)]]

Caused by op 'Reshape_10', defined at:
  File "C:\Anaconda\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Anaconda\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda\envs\tensorflow\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Anaconda\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Anaconda\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "C:\Anaconda\envs\tensorflow\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    

In [ ]:
def conv2_network(x_image, reuse=False): 
    with tf.variable_scope('ConvNet2') as scope: 
        if reuse:
             tf.get_variable_scope().reuse_variables()
        # First Conv and Pooling layers
        W_conv1 = tf.get_variable('c_wconv1',[5,5,1,8],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv1 = tf.get_variable('c_bconv1',[8],initializer=tf.constant_initializer(0))
        h_conv1 = tf.nn.relu(conv_2d(x_image, W_conv1) + b_conv1)
        h_pool1 = avg_pool_2x2(h_conv1)
        
        # Seond Conv and Pooling layers
        W_conv2 = tf.get_variable('d_wconv2',[5,5,8,16],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv2 = tf.get_variable('d_bconv2',[16],initializer=tf.constant_initializer(0))
        h_conv2 = tf.nn.relu(conv_2d(h_pool1, W_conv2) + b_conv2)
        h_pool2 = avg_pool_2x2(h_conv2)
        
        # First fully connected layers
        W_fc1 = tf.get_variable('d_wfc1', [7*7*16,32],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc1 = tf.get_variable('d_bfc1', [32], initializer=tf.constant_initializer(0))
        h_pool_flat = tf.reshape(h_pool2, [-1,7*7*16])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat,W_fc1)+b_fc1)
        
        #Seconf fully connected layers
        W_fc2 = tf.get_variable('d_wfc2', [32,1],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc2 = tf.get_variable('d_bfc2', [1], initializer=tf.constant_initializer(0))

        y_conv = tf.nn.relu(tf.matmul(h_fc1,W_fc2)+b_fc2)
    
    return y_conv
        

In [ ]:
def conv3_network(x_image, reuse=False): 
    with tf.variable_scope('ConvNet3') as scope: 
        if reuse:
             tf.get_variable_scope().reuse_variables()
        # First Conv and Pooling layers
        W_conv1 = tf.get_variable('c_wconv1',[5,5,1,8],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv1 = tf.get_variable('c_bconv1',[8],initializer=tf.constant_initializer(0))
        h_conv1 = tf.nn.relu(conv_2d(x_image, W_conv1) + b_conv1)
        h_pool1 = avg_pool_2x2(h_conv1)
        
        # Seond Conv and Pooling layers
        W_conv2 = tf.get_variable('d_wconv2',[5,5,8,16],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv2 = tf.get_variable('d_bconv2',[16],initializer=tf.constant_initializer(0))
        h_conv2 = tf.nn.relu(conv_2d(h_pool1, W_conv2) + b_conv2)
        h_pool2 = avg_pool_2x2(h_conv2)
        
        # Third Conv and Pooling layers
        W_conv3 = tf.get_variable('d_wconv2',[5,5,16,32],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_conv3 = tf.get_variable('d_bconv2',[32],initializer=tf.constant_initializer(0))
        h_conv3 = tf.nn.relu(conv_2d(h_pool2, W_conv3) + b_conv3)
        h_pool3 = avg_pool_2x2(h_conv3)
        
        # First fully connected layers
        W_fc1 = tf.get_variable('d_wfc1', [7*7*32,64],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc1 = tf.get_variable('d_bfc1', [64], initializer=tf.constant_initializer(0))
        h_pool_flat = tf.reshape(h_pool3, [-1,7*7*32])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat,W_fc1)+b_fc1)
        
        #Seconf fully connected layers
        W_fc2 = tf.get_variable('d_wfc2', [64,1],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b_fc2 = tf.get_variable('d_bfc2', [1], initializer=tf.constant_initializer(0))

        y_conv = tf.nn.relu(tf.matmul(h_fc1,W_fc2)+b_fc2)
    
    return y_conv